In [3]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [4]:
def load_and_preprocess_image(image_path, target_size=(100, 100)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image

def create_pairs(data_directory):
    classes = os.listdir(data_directory)
    pairs = []
    labels = []
    
    for i, class_name in enumerate(classes):
        class_folder = os.path.join(data_directory, class_name)
        images = os.listdir(class_folder)
        
        for img1_name in images:
            img1_path = os.path.join(class_folder, img1_name)
            img1 = load_and_preprocess_image(img1_path)
            
            for j in range(5):  # Number of positive pairs for each anchor image
                # Choose a random image from the same class
                img2_name = random.choice(images)
                img2_path = os.path.join(class_folder, img2_name)
                img2 = load_and_preprocess_image(img2_path)
                
                pairs.append([img1, img2])
                labels.append(1)  # Positive pair
                
                # Choose a random image from a different class
                negative_class = random.choice(classes)
                while negative_class == class_name:
                    negative_class = random.choice(classes)
                
                negative_folder = os.path.join(data_directory, negative_class)
                negative_images = os.listdir(negative_folder)
                img3_name = random.choice(negative_images)
                img3_path = os.path.join(negative_folder, img3_name)
                img3 = load_and_preprocess_image(img3_path)
                
                pairs.append([img1, img3])
                labels.append(0)  # Negative pair
    
    return np.array(pairs), np.array(labels)

def create_siamese_model(input_shape):
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    base_network = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        Flatten(),
        Dense(128, activation='relu')
    ])

    encoded_a = base_network(input_a)
    encoded_b = base_network(input_b)

    # Calculate Euclidean distance between the two encodings
    distance = Lambda(lambda x: K.sqrt(K.sum(K.square(x[0] - x[1]), axis=-1)), output_shape=lambda _: (1,))( 
        [encoded_a, encoded_b])

    siamese_model = Model(inputs=[input_a, input_b], outputs=distance)
    return siamese_model

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

if __name__ == "__main__":
    data_directory = 'Assets'
    pairs, labels = create_pairs(data_directory)

    input_shape = (100, 100, 3)

    siamese_model = create_siamese_model(input_shape)
    siamese_model.compile(optimizer=Adam(learning_rate=0.001), loss=contrastive_loss)

    img1 = pairs[:, 0]
    img2 = pairs[:, 1]

    # Reshape img1 and img2 to match the model input requirements
    img1 = img1.reshape(-1, 100, 100, 3)
    img2 = img2.reshape(-1, 100, 100, 3)

    siamese_model.fit([img1, img2], labels, batch_size=32, epochs=10)

    model_path = 'siamese.h5'
    siamese_model.save(model_path)
    print("Model saved successfully.")


c:\Users\GDIT\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 27s 479ms/step - loss: 1.6832
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 483ms/step - loss: 0.4908
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 27s 498ms/step - loss: 0.4968
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 480ms/step - loss: 0.4979
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 484ms/step - loss: 0.5178
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 479ms/step - loss: 0.5297
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 477ms/step - loss: 0.4821
Epoch 8/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 476ms/step - loss: 0.5032
Epoch 9/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 476ms/step - loss: 0.5114
Epoch 10/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 477ms/step - loss: 0.4993


Model saved successfully.


In [17]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Lambda
from tensorflow.keras import backend as K  # Import Keras backend module
from mtcnn import MTCNN
from model import load_and_preprocess_image

In [18]:
# Initialize the MTCNN detector
detector = MTCNN()

# Load the Siamese model
siamese_model = load_model('siamese.h5', compile=False)  # Load model without compiling

# Load and preprocess the image
image = cv2.cvtColor(cv2.imread("image1.jpg"), cv2.COLOR_BGR2RGB)

# Detect faces and facial keypoints
result = detector.detect_faces(image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [20]:

if result:
    # Load classmates' known images (for comparison)
    classmates = ['Abdullah(ABD)', 'Abdur Rehman Durani', 'Abdur Rehman Sajid', 'Adeen Amir', 'Affan Ali Khan', 'Ahmad Ali Abid', 'Ahmad Fareed sukhera', 'Ali Inayat', 'Arsal Sheikh',
                   'Basim Mehmood', 'Eman Anjum', 'Faizan Haq', 'Farwa Toor', 'Hammad Anwar', 'Hamza Ahmed Zuberi', 'Hamza Wajid', 'Haya Noor', 'Itba Malahat', 'Lailoma Noor', 'Mia Akbar Jaan',
                     'Mujtaba', 'Omar Khan', 'Raja', 'Rehan Riaz', 'Saadullah', 'Sameer Shehzad', 'Sheharyar Sadiq', 'Sherry', 'Syed Ibrahim Hamza', 'Talha Wajid', 'Tehrim Ahmed',
                       'Umair', 'Umer Tayyab', 'Zaid Bin Muzammil', 'Zaid Dandia' ]  # List of classmate names (corresponding to folder names)
    known_embeddings = {}

    for class_name in classmates:
        image_path = f"Assets/{class_name}/sample.png"  # Use a sample image for each classmate
        img = load_and_preprocess_image(image_path)  # Preprocess image
        embedding = siamese_model.predict([np.expand_dims(img, axis=0), np.expand_dims(img, axis=0)])
        known_embeddings[class_name] = embedding

    # Loop through each detected face
    for face_data in result:
        bounding_box = face_data['box']  

        # Extract face region
        face_img = image[bounding_box[1]:bounding_box[1] + bounding_box[3],
                         bounding_box[0]:bounding_box[0] + bounding_box[2]]
        
        # Preprocess extracted face image
        face_img = cv2.resize(face_img, (100, 100))  # Resize to match Siamese model input size
        face_img = face_img.astype(np.float32) / 255.0  # Normalize

        # Obtain face embedding using the Siamese model
        face_embedding = siamese_model.predict([np.expand_dims(face_img, axis=0), np.expand_dims(face_img, axis=0)])

        # Compare face embedding with known embeddings to recognize the face
        min_distance = float('inf')
        recognized_classmate = None

        for class_name in known_embeddings:
            distance = np.linalg.norm(face_embedding - known_embeddings[class_name])
            if distance < min_distance:
                min_distance = distance
                recognized_classmate = class_name

        # Draw bounding box around the face with recognized label
        cv2.rectangle(image,
                      (bounding_box[0], bounding_box[1]),
                      (bounding_box[0] + bounding_box[2], bounding_box[1] + bounding_box[3]),
                      (0, 155, 255),
                      2)
        cv2.putText(image, recognized_classmate, (bounding_box[0], bounding_box[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 155, 255), 2)

    # Display the annotated image with recognized faces
    cv2.imshow("Recognized Images", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    print("No faces detected in the image.")

NameError: Exception encountered when calling Lambda.call().

[1mname 'K' is not defined[0m

Arguments received by Lambda.call():
  • inputs=['tf.Tensor(shape=(1, 128), dtype=float32)', 'tf.Tensor(shape=(1, 128), dtype=float32)']
  • mask=['None', 'None']
  • training=False

NotImplementedError: Exception encountered when calling Lambda.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Lambda.call():
  • args=(['<KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_71>', '<KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_73>'],)
  • kwargs={'mask': ['None', 'None']}